# Chapter 9 - Rag based applications

Building a chatbot using RAG

## What is RAG

## QandA with RAG


### Extract text from a webpage

In [37]:
from bs4 import BeautifulSoup
import requests
import re

url = "https://www.gutenberg.org/cache/epub/64317/pg64317-images.html" 

response = requests.get(url)
page_html = response.text

soup = BeautifulSoup(page_html, "html.parser")
a    = soup.find('div', attrs={"class" : "container"})
text = a.parent.parent.get_text()
start = re.escape("*** START OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***")
end = re.escape("*** END OF THE PROJECT GUTENBERG EBOOK")
text = re.search('{}(.*){}'.format(start, end), text, re.S).group(1)



### Create documents to index

In [45]:
from langchain.docstore.document import Document

doc =  Document(page_content=text, metadata={"source": "local"})


In [47]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents([doc])


Document(page_content='The Great GatsbybyF. Scott Fitzgerald\n\nTable of Contents\n\n\nI\n\n\nII\n\n\nIII\n\n\nIV\n\n\nV\n\n\nVI\n\n\nVII\n\n\nVIII\n\n\nIX\n\n\n\n\n\r\nOnce again\r\nto\r\nZelda\r\n\n\n\n\n\nThen wear the gold hat, if that will move her;\n\nIf you can bounce high, bounce for her too,\n\nTill she cry “Lover, gold-hatted, high-bouncing lover,\n\nI must have you!”\n\n\nThomas Parke d’Invilliers\n\n\n\n\nI\n\r\nIn my younger and more vulnerable years my father gave me some advice that I’ve been turning over in my mind ever since.\r\n\n\r\n“Whenever you feel like criticizing anyone,” he told me, “just remember that all the people in this world haven’t had the advantages that you’ve had.”', metadata={'source': 'local', 'start_index': 2})

### Index the documents

In [48]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# Equivalent to SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)

## Similarity based retrieval

In [49]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [50]:
retrieved_docs = retriever.invoke("")
print(retrieved_docs[0].page_content)

With fenders spread like wings we scattered light through half Astoria—only half, for as we twisted among the pillars of the elevated I heard the familiar “jug-jug-spat!” of a motorcycle, and a frantic policeman rode alongside.


“All right, old sport,” called Gatsby. We slowed down. Taking a white card from his wallet, he waved it before the man’s eyes.


“Right you are,” agreed the policeman, tipping his cap. “Know you next time, Mr. Gatsby. Excuse me!”


“What was that?” I inquired. “The picture of Oxford?”


“I was able to do the commissioner a favour once, and he sends me a Christmas card every year.”


### Q&A Text Generation

In [73]:
from langchain_core.prompts import PromptTemplate
import torch
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# HuggingFace Pipeline
generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True)

## Langchain pipeline
llm = HuggingFacePipeline(pipeline=generate_text)

template = """You are an assistant for question answering task.Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

%time rag_chain.invoke("Where did the accident happen?")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CPU times: user 4min 57s, sys: 15.7 s, total: 5min 13s
Wall time: 53.7 s


'It happened, and that’s all I know.'

## Chat with History

In [68]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage


contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever | format_docs
    )
    | qa_prompt
    | llm
)

In [69]:
chat_history = []

question = "Who is Daisy?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg])

second_question = "How did she die?"
rag_chain.invoke({"question": second_question, "chat_history": chat_history})


'Tom Buchanan was in love with Daisy for several years before he knew that she was involved in some criminal activity. At first, he suspected nothing, but one day he saw Daisy leave her home with another woman and he discovered that Daisy had a driver’s license. When Tom confronted Daisy about it, she attempted to run away and tore open the back of her neck, killing her instantly.'

In [72]:
import gradio as gr

def echo(message, history):
    question = message
    ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
    chat_history.extend([HumanMessage(content=question), ai_msg])
    return ai_msg


demo = gr.ChatInterface(fn=echo, examples=["hello", "hola", "merhaba"], title="The Great Gatsby Bot")
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


## Dolly Example

In [51]:
import torch
from transformers import pipeline

generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="cpu", return_full_text=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

# template for an instrution with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

# template for an instruction with input
prompt_with_context = PromptTemplate(
    input_variables=["instruction", "context"],
    template="{instruction}\n\nInput:\n{context}")

hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt)
llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)


In [7]:
print(llm_chain.predict(instruction="Explain to me the difference between nuclear fission and fusion.").lstrip())


Fission breaks down an atom into smaller atoms, while fusion together the smaller atomic nuclei to form one larger atomic nucleus.


In [8]:
context = """George Washington (February 22, 1732[b] - December 14, 1799) was an American military officer, statesman,
and Founding Father who served as the first president of the United States from 1789 to 1797."""

print(llm_context_chain.predict(instruction="When was George Washington president?", context=context).lstrip())


George Washington was president from 1789 to 1797.




    Improve the quality of the embeddings: You're currently using the sentence-transformers/all-mpnet-base-v2 model for embeddings. While this is a good general-purpose model, you might get better results with a model that's more specialized for your specific task. For example, you could try using a model that's been fine-tuned on a dataset of similar documents to your operation manual.

    Optimize the search: You're currently using the FAISS vector store for similarity search. While FAISS is a good choice for large-scale similarity search, it might not be the most efficient choice for your specific task. You could try using a different vector store, such as Pinecone or Weaviate, to see if they provide better performance.

    Optimize the text splitting: You're currently splitting the text into chunks of 1000 characters with an overlap of 200 characters. This might be too fine-grained, resulting in a lot of redundant computations. You could try increasing the chunk size and reducing the overlap to improve efficiency.

    Optimize the prompt: You're currently using a generic prompt template. You might get better results by customizing the prompt to better match the style and content of your operation manual.


In [1]:
import pandas as pd


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 677, in start
    s

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 677, in start
    s

AttributeError: _ARRAY_API not found